In [1]:
import numpy as np
np.random.seed(101)
import pandas as pd

In [4]:
# generate data for 10 tenants
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
tenants = ['tenant0', 'tenant1', 'tenant2', 'tenant3', 'tenant4', 'tenant5', 'tenant6', 'tenant7', 'tenant8', 'tenant9']
idx = pd.MultiIndex.from_product([months, tenants], names=['Month', 'Tenant_name'])
records = pd.DataFrame(np.random.randint(100, 10000, size=(120, 9)), index=idx, columns=['House_no','Rent', 'Water_units_used',
                                                                                        'Water_bill', 'Garbage_bill',
                                                                                        'Security_fee', 'Total_amount_payable',
                                                                                        'Total_amount_paid','Total_amount_due'])
records.head()

House_no  Rent  Water_units_used  Water_bill  Garbage_bill  \
Month Tenant_name                                                               
Jan   tenant0          7321  6305              3024         413          7467   
      tenant1          2345  4339              9737         793          7177   
      tenant2          5563  2814              7637        4959          4180   
      tenant3          4098  5700              4159        5945          8012   
      tenant4          2419  2561              2231        2393          5829   

                   Security_fee  Total_amount_payable  Total_amount_paid  \
Month Tenant_name                                                          
Jan   tenant0              2023                  3063               5025   
      tenant1              7208                  4539               2933   
      tenant2              4097                  3478                251   
      tenant3              7110                  9793                308   
      tenant4              7393                  4310               3878   

                   Total_amount_due  
Month Tenant_name                    
Jan   tenant0                  2973  
      tenant1                  5775  
      tenant2                  2819  
      tenant3                  9878  
      tenant4                  5181

In [5]:
records_flat = records.reset_index()
records_flat.head()

,Month,Tenant_name,House_no,Rent,Water_units_used,Water_bill,Garbage_bill,Security_fee,Total_amount_payable,Total_amount_paid,Total_amount_due
0,Jan,tenant0,7321,6305,3024,413,7467,2023,3063,5025,2973
1,Jan,tenant1,2345,4339,9737,793,7177,7208,4539,2933,5775
2,Jan,tenant2,5563,2814,7637,4959,4180,4097,3478,251,2819
3,Jan,tenant3,4098,5700,4159,5945,8012,7110,9793,308,9878
4,Jan,tenant4,2419,2561,2231,2393,5829,7393,4310,3878,5181


In [6]:
records.columns

Index(['House_no', 'Rent', 'Water_units_used', 'Water_bill', 'Garbage_bill',
       'Security_fee', 'Total_amount_payable', 'Total_amount_paid',
       'Total_amount_due'],
      dtype='object')

In [7]:
def actualize_data(df):
    # set seed
    np.random.seed(101)
    
    # flatten columns
    df = df.reset_index()
    
    # restructure data
    df['House_no'] = df['Tenant_name'].apply(lambda x: x.partition('tenant')[-1])
    df['Rent'] = 10000
    df['Water_units_used'] = np.random.randint(1, 9, 120)
    df['Water_bill'] = df.Water_units_used * 110 # price per unit
    df['Garbage_bill'] = 150
    df['Security_fee'] = 500
    df['Total_amount_payable'] = df[['Rent', 'Water_bill', 'Garbage_bill',
                                     'Security_fee']].agg(func=sum, axis=1)
    df['Total_amount_paid'] = df.Total_amount_payable - np.round(np.random.randint(500, 3000, size=120), decimals=-2)
    df['Total_amount_due'] = df.Total_amount_payable - df.Total_amount_paid
    
    df = pd.pivot_table(df, index=['Month', 'Tenant_name'])
    df = df[['Rent', 'Water_units_used', 'Water_bill', 'Garbage_bill', 'Security_fee', 'Total_amount_payable',
             'Total_amount_paid', 'Total_amount_due']]
    df = df.reindex(index=idx)
    return df

In [8]:
actual_records = actualize_data(records)
actual_records

Rent  Water_units_used  Water_bill  Garbage_bill  \
Month Tenant_name                                                      
Jan   tenant0      10000                 8         880           150   
      tenant1      10000                 4         440           150   
      tenant2      10000                 2         220           150   
      tenant3      10000                 7         770           150   
      tenant4      10000                 8         880           150   
...                  ...               ...         ...           ...   
Dec   tenant5      10000                 4         440           150   
      tenant6      10000                 4         440           150   
      tenant7      10000                 3         330           150   
      tenant8      10000                 2         220           150   
      tenant9      10000                 7         770           150   

                   Security_fee  Total_amount_payable  Total_amount_paid  \
Month Tenant_name                                                          
Jan   tenant0               500                 11530               8530   
      tenant1               500                 11090              10090   
      tenant2               500                 10870              10370   
      tenant3               500                 11420              10020   
      tenant4               500                 11530               8630   
...                         ...                   ...                ...   
Dec   tenant5               500                 11090              10490   
      tenant6               500                 11090               8490   
      tenant7               500                 10980               8180   
      tenant8               500                 10870               8970   
      tenant9               500                 11420               9120   

                   Total_amount_due  
Month Tenant_name                    
Jan   tenant0                  3000  
      tenant1                  1000  
      tenant2                   500  
      tenant3                  1400  
      tenant4                  2900  
...                             ...  
Dec   tenant5                   600  
      tenant6                  2600  
      tenant7                  2800  
      tenant8                  1900  
      tenant9                  2300  

[120 rows x 8 columns]

In [9]:
# I have no idea why TF the House_no column is being designated as NaNs in the actualize_data function
# This is why it is not indexed to be returned at the end of this function

In [10]:
# save records to csv
actual_records.to_csv("records.csv")

In [11]:
# reading csv file as a multiindex dataframe
df = pd.read_csv("records.csv", index_col=['Month', 'Tenant_name'])
df

Rent  Water_units_used  Water_bill  Garbage_bill  \
Month Tenant_name                                                      
Jan   tenant0      10000                 8         880           150   
      tenant1      10000                 4         440           150   
      tenant2      10000                 2         220           150   
      tenant3      10000                 7         770           150   
      tenant4      10000                 8         880           150   
...                  ...               ...         ...           ...   
Dec   tenant5      10000                 4         440           150   
      tenant6      10000                 4         440           150   
      tenant7      10000                 3         330           150   
      tenant8      10000                 2         220           150   
      tenant9      10000                 7         770           150   

                   Security_fee  Total_amount_payable  Total_amount_paid  \
Month Tenant_name                                                          
Jan   tenant0               500                 11530               8530   
      tenant1               500                 11090              10090   
      tenant2               500                 10870              10370   
      tenant3               500                 11420              10020   
      tenant4               500                 11530               8630   
...                         ...                   ...                ...   
Dec   tenant5               500                 11090              10490   
      tenant6               500                 11090               8490   
      tenant7               500                 10980               8180   
      tenant8               500                 10870               8970   
      tenant9               500                 11420               9120   

                   Total_amount_due  
Month Tenant_name                    
Jan   tenant0                  3000  
      tenant1                  1000  
      tenant2                   500  
      tenant3                  1400  
      tenant4                  2900  
...                             ...  
Dec   tenant5                   600  
      tenant6                  2600  
      tenant7                  2800  
      tenant8                  1900  
      tenant9                  2300  

[120 rows x 8 columns]

In [45]:
# records for a specific tenant
records.reset_index()[records.reset_index().Tenant_name == 'tenant0'].pivot_table(index='Month').reindex(months)

,Garbage_bill,House_no,Rent,Security_fee,Total_amount_due,Total_amount_paid,Total_amount_payable,Water_bill,Water_units_used
Month,,,,,,,,,
Jan,7467,7321,6305,2023,2973,5025,3063,413,3024
Feb,9255,129,879,7236,6924,2677,7721,4004,710
Mar,3432,896,5796,1585,3758,9912,6803,7797,8411
Apr,1087,7462,3708,8391,8928,3854,1536,6037,5747
May,7859,4330,9701,1901,7983,5942,5252,889,5578
Jun,6317,6439,5602,9273,6000,6315,5151,7637,3504
Jul,8395,9571,9187,3519,2686,5396,738,4669,3359
Aug,7092,227,415,5502,8625,5223,5641,1619,9259
Sep,2333,2780,5948,2017,2272,8537,4558,7079,2550


## **More data output statistics**

In [37]:
pd.DataFrame(df.reset_index().groupby('Month')['Total_amount_paid'].sum()).T

Month,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
Total_amount_paid,91190,95780,93220,90550,95170,94140,96300,93300,91570,97910,91930,92650


In [39]:
# Monthly total amounts statistics
pd.DataFrame(df.reset_index().groupby('Month')[['Total_amount_payable', 'Total_amount_paid', 'Total_amount_due']].sum()).T

Month,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
Total_amount_payable,110790,110680,111120,111450,112770,113540,112000,110900,111670,113210,111230,111450
Total_amount_paid,91190,95780,93220,90550,95170,94140,96300,93300,91570,97910,91930,92650
Total_amount_due,19600,14900,17900,20900,17600,19400,15700,17600,20100,15300,19300,18800


In [41]:
# End year statistics (totals for all months)
pd.DataFrame(df.reset_index().groupby('Month')[['Total_amount_payable', 'Total_amount_paid', 'Total_amount_due']].sum()).T.agg(sum, axis=1)

Total_amount_payable    1340810
Total_amount_paid       1123710
Total_amount_due         217100
dtype: int64